In [ ]:
from google.colab import drive
drive.mount('/gdrive')
%cd /gdrive

from google.colab import drive
drive.mount('/content/drive')

import random
import cv2
from matplotlib import pyplot as plt
import albumentations as A
import pandas as pd
import os
import json

Mounted at /gdrive
/gdrive
Mounted at /content/drive


In [ ]:
JSON = "/content/drive/My Drive/thyroid_nodule_detection/Freehand to Bounding Box Conversion Fix/fixed_labels/cropped_dataset/validation/cropped_labels.json"
IMAGE_DIR = "/content/drive/My Drive/thyroid_nodule_detection/Freehand to Bounding Box Conversion Fix/fixed_labels/cropped_dataset/validation/cropped_images"

In [ ]:
temp_df = pd.read_json(JSON)
print(temp_df.head())

                                               cases
0  {'case_id': '399_1', 'bboxes': [[174.0, 105.00...
1  {'case_id': '398_1', 'bboxes': [[184.0, 80.0, ...
2  {'case_id': '400_1', 'bboxes': [[127.0, 79.0, ...
3  {'case_id': '392_1', 'bboxes': [[169.0, 70.0, ...
4  {'case_id': '394_1', 'bboxes': [[66.0, 28.9999...


In [ ]:
!pip install bbaug

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
from bbaug.policies.policies import POLICY_TUPLE
from bbaug import policies

aug_policy = policies.policies_v0()
print(aug_policy)
aug_policy1 = policies.policies_v1()
print(aug_policy1)
aug_policy2 = policies.policies_v2()
print(aug_policy2)
aug_policy3 = policies.policies_v3()
print(aug_policy3)

# instantiate the policy container with the selected policy set
# policy_container = policies.PolicyContainer(aug_policy)

custom_policy = [
    [POLICY_TUPLE('Rotate', 1.0, 10), POLICY_TUPLE('Color', 1.0, 6)]
]

print(custom_policy)

policy_container = policies.PolicyContainer(custom_policy)


[[policy(name='Translate_X', probability=0.6, magnitude=4), policy(name='Equalize', probability=0.8, magnitude=10)], [policy(name='Translate_Y_BBox', probability=0.2, magnitude=2), policy(name='Cutout', probability=0.8, magnitude=8)], [policy(name='Sharpness', probability=0.0, magnitude=8), policy(name='Shear_X', probability=0.4, magnitude=0)], [policy(name='Shear_Y', probability=1.0, magnitude=2), policy(name='Translate_Y_BBox', probability=0.6, magnitude=6)], [policy(name='Rotate', probability=0.6, magnitude=10), policy(name='Color', probability=1.0, magnitude=6)]]
[[policy(name='Translate_X', probability=0.6, magnitude=4), policy(name='Equalize', probability=0.8, magnitude=10)], [policy(name='Translate_Y_BBox', probability=0.2, magnitude=2), policy(name='Cutout', probability=0.8, magnitude=8)], [policy(name='Sharpness', probability=0.0, magnitude=8), policy(name='Shear_X', probability=0.4, magnitude=0)], [policy(name='Shear_Y', probability=1.0, magnitude=2), policy(name='Translate_Y

In [ ]:
def process_df(df):
    cases = df
    case_list = []
    for case in df['cases']:
        image_id = case['case_id'] + ".jpg"
    #     id = case['tirads']
    #     if len(id) == 2:
    #       id = 4
    #     else:
    #       id = int(id)
        
    #     lenght_of_array = len(case['bboxes'])
    #     labels = lenght_of_array * [id]
        
        bboxes = []
        for bbox in case['bboxes']:
          bbox_array = []
          xmin = bbox[0]
          bbox_array.append(xmin)
          ymin = bbox[1]
          bbox_array.append(ymin)
          xmax = bbox[0] + bbox[2]
          bbox_array.append(xmax)
          ymax = bbox[1] + bbox[3]
          bbox_array.append(ymax)
          # xmin = bbox['x']
          # bbox_array.append(xmin)
          # ymin = bbox['y']
          # bbox_array.append(ymin)
          # xmax = bbox['w'] + bbox['x']
          # bbox_array.append(xmax)
          # ymax = bbox['h'] + bbox['y']
          # bbox_array.append(ymax)

          bboxes.append(bbox_array)

        temp_case = {}
        temp_case['case_id'] = case['case_id']
        temp_case['image'] = cv2.imread(os.path.join(IMAGE_DIR, image_id), cv2.IMREAD_COLOR)
        temp_case['labels'] = case['labels']
        temp_case['bboxes'] = bboxes
        case_list.append(temp_case)
    return pd.DataFrame(case_list)

In [60]:
temp_df = pd.read_json(JSON)
df = process_df(temp_df)
df.head()
print(df)

original: [174.0, 105.00000000000001, 50.00000000000003, 58.999999999999986]
new: [174.0, 105.00000000000001, 224.00000000000003, 164.0]
original: [195.0, 151.0, 59.99999999999997, 106.0]
new: [195.0, 151.0, 254.99999999999997, 257.0]
original: [184.0, 80.0, 92.0, 131.0]
new: [184.0, 80.0, 276.0, 211.0]
original: [127.0, 79.0, 170.0, 147.00000000000003]
new: [127.0, 79.0, 297.0, 226.00000000000003]
original: [169.0, 70.0, 56.0, 95.0]
new: [169.0, 70.0, 225.0, 165.0]
original: [66.0, 28.999999999999996, 264.0, 194.0]
new: [66.0, 28.999999999999996, 330.0, 223.0]
original: [140.0, 109.0, 155.0, 75.0]
new: [140.0, 109.0, 295.0, 184.0]
original: [155.0, 81.0, 144.0, 111.0]
new: [155.0, 81.0, 299.0, 192.0]
original: [217.0, 122.0, 99.0, 81.0]
new: [217.0, 122.0, 316.0, 203.0]
original: [180.0, 57.0, 88.0, 129.00000000000003]
new: [180.0, 57.0, 268.0, 186.00000000000003]
original: [122.0, 46.99999999999999, 163.0, 103.0]
new: [122.0, 46.99999999999999, 285.0, 150.0]
  case_id                

In [61]:
def data_augmentation():
  images = []
  bboxes = []
  # for index,row in df.iterrows():
  for ind in df.index:
    # select a random policy from the policy set
    policy = policy_container.select_random_policy() 
    #policy = custom_policy
    #img_aug, bbs_aug = policy_container.apply_augmentation(policy, row.get('image'), row.get('bboxes'), row.get('labels'))
    image = df['image'][ind]
    bbox_array = df['bboxes'][ind]
    label_array = df['labels'][ind]
    img_aug, bbs_aug = policy_container.apply_augmentation(policy, image, bbox_array, label_array)
    images.append(img_aug)
    bboxes.append(bbs_aug)

  return images,bboxes;

In [ ]:
from PIL import Image as im

In [64]:
def image_convert(image):
    image = image.clone().cpu().numpy()
    image = image.transpose((1, 2, 0))
    image = (image * 255).astype(np.uint8)
    return image

def plot_img(image_array, bbox):
    # vis0 = cv2.fromarray(image_array)
    # out = image_array[:, :, ::-1].copy()
    # out = cv2.cvtColor(image_array, cv2.)
    # image = image_convert(out)

    #image = np.ascontiguousarray(image_array)
    
    for i in bbox:
        print(f'bbox in plot_img {i}')
        cv2.rectangle(image_array, (int(i[0]), int(i[1])), (int(i[2]), int(i[3])), (0, 255, 0), thickness=2)
    plt.figure(figsize=(10, 10))
    plt.imshow(image_array)
    plt.show()

In [ ]:
import numpy as np
images_aug,bboxes_aug = data_augmentation()

# im = image_convert(images_aug[1])
# plt.imshow(images_aug[1])

for i in range(8):
  
  original_bboxes = [] # in xmin, ymin, xmax, ymax format
  for bbox in df.get('bboxes').get(i):
    converted_bbox = []
    converted_bbox.append(bbox[0])
    converted_bbox.append(bbox[1])
    converted_bbox.append(bbox[0] + bbox[2])
    converted_bbox.append(bbox[1] + bbox[3])
    original_bboxes.append(converted_bbox)

  plot_img(df.get('image').get(i), original_bboxes)
  plot_img(images_aug[i], bboxes_aug[i])
